# Home assignment 2

You should work on the assignement in groups of 2 participants. 

Upload your solution as a jupyter notebook to L2P by 26th of June 23:59h. (The deadline is strict)

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results.

# 1. Dynamic PageRank
Consider a random walk setting where the transistion matrix changes over time. At any point of time the probability of a random surfer to jump to a linked page is proportional to the number of previous visits. To start with all the pages are equally likely to be chosen but as the walk continues and the nodes are visited the transition probability changes as proportional to number of previous visits. For example let a page 'a' is linked to pages 'b', 'c' and 'd'. The random surfer currently resides at 'a' and the pages 'b', 'c' and 'd' have already been visited 5, 3 and 2 times respectively. The transition probability would be 0.5, 0.3 and 0.2 respectively. As a new node is viited the probabilities change. The random surfer continues to surf with probability 0.8. Generate 100 random walks and rank the nodes based on the frequency of visit. The random walk should be performed on a drected Erdos-Renyi graph with number of nodes n=200 and probability of edge creation p = 0.4. 

Hint: Use networkx library for generating graph.
    

In [9]:
import networkx as nx
import numpy as np

walks = 100
c_p = 0.8

freq_visit = [1 for i in range(200)]
G = nx.erdos_renyi_graph(200, 0.4, directed=True)

In [10]:
while walks>0:
    start = np.random.randint(200) # randomly select a start node
    l = 0
    while True:
        freq_visit[start]+=1
        nbrs = [nbr for nbr in G.neighbors(start)]
        trans_prob = [freq_visit[node] for node in nbrs]
        sum_ = sum(trans_prob)
        trans_prob = [val/sum_ for val in trans_prob]
        nxt = np.random.choice(nbrs,p=trans_prob)
        start = nxt
        if np.random.uniform(0,1)>0.8:
            break

    walks-=1


sorted(range(len(freq_visit)), key=lambda k: freq_visit[k], reverse=True)[:10]

[143, 4, 99, 110, 193, 68, 100, 115, 119, 133]

# 2. Recommendation
a. Compare the recommendation algorithms (SVD, NMF, Baseline, k-NN and Random) available in surprise package on movielens dataset in terms of RMSE and MAE.

b. Consider the movielens dataset and divide it into (i) training set with 50% of the data (train the algorithms on this part) and (ii) 25% validation set and (iii) test set with the rest. Estimate the ratings of the test set using the algorithms (same as in a) provided by the package on the training set. Your final rating should be weighted average of the ratings predicted by the algorithms. The weights should be learnt on the validation set. Performance should be measured in terms of RMSE.

Hint: Use grid search/step-wise update like SGD for learning the weights

In [13]:
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.random_pred import NormalPredictor
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.baseline_only import BaselineOnly

In [40]:
data = Dataset.load_builtin('ml-100k')
algorithms = [SVD(),NMF(),NormalPredictor(),KNNBasic(),BaselineOnly()]
for alg in algorithms:
    cross_validate(alg, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9348  0.9424  0.9309  0.9374  0.9337  0.9359  0.0039  
MAE (testset)     0.7363  0.7429  0.7339  0.7399  0.7374  0.7381  0.0031  
Fit time          3.53    3.55    3.55    3.55    3.55    3.55    0.01    
Test time         0.09    0.23    0.09    0.09    0.09    0.12    0.05    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9646  0.9610  0.9584  0.9631  0.9621  0.9618  0.0021  
MAE (testset)     0.7602  0.7547  0.7522  0.7577  0.7560  0.7562  0.0027  
Fit time          3.77    3.79    3.79    4.08    4.06    3.90    0.14    
Test time         0.08    0.08    0.08    0.22    0.08    0.11    0.06    
Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
R

In [16]:
train,rest = train_test_split(data,test_size=0.5)

In [29]:
# splitting rest to test and validate
import random
random.shuffle(rest)
n = int(len(rest)/2)
validate = rest[:n]
test = rest[n:-1]

In [96]:
# training algorithms on the training set
for alg in algorithms:
    alg.fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [97]:
lr = 0.001
W = [np.random.uniform(0.1) for i in range(len(algorithms))]
for ins in validate:
    out = [alg.test([ins])[0].est for alg in algorithms]
    rating = ins[2]
    pred_rating = np.dot(W,out)
    error = rating - pred_rating
    W = W + np.multiply(2*error*lr,out)

In [106]:
RMSE = 0.0
for ins in test:
    out = [alg.test([ins])[0].est for alg in algorithms]
    pred_rating = np.dot(W,out)
    rating = ins[2]
    #print ((pred_rating - rating)**2)
    RMSE+= (rating - pred_rating)**2  
RMSE = RMSE/len(test)
print (np.sqrt(RMSE))

1.03128100137997


# 3. Hidden Markov model
Consider the HMM package https://hmmlearn.readthedocs.io/en/latest/

a. Generate sequences with multinomial HMM (2 symbols and 4 hidden states) and given parameters. Start probability - {0.4,0.2,0.1,0.3}, Transition matrix - {{0.2,0.3,0.1,0.4},{0.3,0.3,0.2,0.2},{0.4,0.2,0.3,0.1},{0.2,0.3,0.1,0.4}}, Emission probability - {{0.2,0.8},{0.1,0.9},{0.5,0.5},{0.6,0.4}}.


b. Consider a sequence - {1 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1}. Fit a multinomial HMM considering 4 states and obtain hidden state w


In [109]:
from hmmlearn import hmm
import numpy as np

In [110]:
from hmmlearn.hmm import MultinomialHMM
model = MultinomialHMM(n_components=4)

In [41]:
model.startprob_ = np.array([0.4, 0.2, 0.1,0.3])

In [47]:
model.emissionprob_ = np.array([[0.2,0.8],[0.1,0.9],[0.5,0.5],[0.6,0.4]])

In [48]:
model.transmat_ = np.array([[0.2,0.3,0.1,0.4],[0.3,0.3,0.2,0.2],[0.4,0.2,0.3,0.1],[0.2,0.3,0.1,0.4]])

In [61]:
X, Z = model.sample(20)

In [53]:
Z

array([3, 1, 0, 1, 2, 0, 1, 3, 3, 3, 1, 0, 1, 0, 0, 2, 0, 0, 1, 0])

In [112]:
X = np.array([1,0,0,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,1,0,1]).reshape(-1,1)

In [113]:
model2 = MultinomialHMM(n_components=4)
model2.fit(X)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=4,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x10b4698b8>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [66]:
model2.predict(X)

array([2, 3, 1, 1, 2, 3, 0, 2, 3, 0, 1, 1, 1, 2, 3, 0, 1, 1, 2, 3])

In [69]:
model2.score_samples(X)

(-12.949709996850093,
 array([[3.10548214e-05, 2.86953190e-01, 7.13015755e-01, 1.71229469e-16],
        [3.49782921e-01, 9.23456361e-02, 5.97809891e-02, 4.98090454e-01],
        [1.88020845e-01, 4.13964540e-01, 3.90119840e-01, 7.89477545e-03],
        [1.49952097e-01, 4.24396437e-01, 4.16710604e-01, 8.94086190e-03],
        [1.01732519e-01, 3.95376788e-01, 4.92028558e-01, 1.08621356e-02],
        [2.60476098e-01, 9.54190839e-02, 7.62377150e-02, 5.67867103e-01],
        [4.58053362e-01, 1.02150744e-01, 6.22004563e-02, 3.77595437e-01],
        [1.20561368e-01, 3.97179530e-01, 4.73751921e-01, 8.50718147e-03],
        [2.61125441e-01, 9.70595058e-02, 7.77086419e-02, 5.64106411e-01],
        [4.49075723e-01, 1.00599489e-01, 6.19533663e-02, 3.88371422e-01],
        [1.75914395e-01, 4.18389541e-01, 3.98289942e-01, 7.40612222e-03],
        [1.49217168e-01, 4.22164754e-01, 4.19228412e-01, 9.38966519e-03],
        [1.51306768e-01, 4.23525491e-01, 4.15978843e-01, 9.18889737e-03],
        [1.01753

# 4. PrefixSpan 
Implement the Prefix algorithm for Sequential Pattern Mining.